In [7]:
from splinter import Browser
from bs4 import BeautifulSoup
import shutil

In [8]:
# Set the zip codes
codes = []
codes.append('27606\n')   # just hard code one in
code = codes[0]    # set the variable for zip_and_click

In [9]:
# inititial website to visit
url = 'https://weather.com'

In [10]:
def chrome():
    '''
    Finds the chromedriver in the system and creates a Chrome browser
    '''
    executable_path = {'executable_path': shutil.which('chromedriver')}
    browser = Browser('chrome', **executable_path)
    return(browser)

In [11]:
browser = chrome()

In [12]:
browser.visit(url)

In [13]:
today_text = browser.find_by_text('Today')

In [14]:
len(today_text)

1

In [16]:
today_text.value

'Today'

In [17]:
inputs = browser.find_by_tag('input')     # get the search box reference
search_box = inputs[0] # 
if inputs[0]:
    print('got input box')
else:
    print('did not get it')

got input box


In [18]:
len(inputs)

1

In [19]:
search_box.fill('27606')

In [20]:
code_text = browser.find_by_text('27606')

In [21]:
if code_text:
    print('found the entered code')
else:
    print('did not find it')

did not find it


In [ ]:
### SEARCH FOR THE TABLE THAT HOLDS THE SEARCH RESULTS ###